# StepFun 多模态推理与 Reasoning 获取示例

本 Notebook 演示了如何使用 StepFun 平台提供的 `OpenAI` 兼容接口，调用多模态模型（如 `step-3`）进行图片分析，并展示如何获取模型在流式输出中的**推理过程 (`reasoning`)**。

参考文档：[https://platform.stepfun.com/docs/guide/reasoning](https://platform.stepfun.com/docs/guide/reasoning#%E8%8E%B7%E5%8F%96-reasoning-%E5%86%85%E5%AE%B9)

### 前置条件

1.  **安装依赖：** `pip install openai`
2.  **设置环境变量：** 确保您的运行环境中已设置 `STEPFUN_API_KEY` 和 `STEPFUN_ENDPOINT`。
3.  **图片文件：** 请在 Notebook 所在目录创建 `./media/` 文件夹，并放入一个名为 `test-监测.png` 的图片文件用于测试。


In [ ]:
from openai import OpenAI
import time
import base64
import os
import sys


BASE_URL = "https://api.stepfun.com/v1"
STEP_API_KEY = os.environ["STEPFUN_API_KEY"]

# --- 配置 --- 

# 初始化 OpenAI 客户端
client = OpenAI(api_key=STEP_API_KEY, base_url=BASE_URL)

# 必填-选择模型
COMPLETION_MODEL = "step-3"
#COMPLETION_MODEL = "step-2-mini"


def image_to_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# 注意提供准确的图片路径
image_path = "./media/01_鱼香肉丝.jpg"
bstring = image_to_base64(image_path)


user_prompt = "帮我看看这是什么菜，如何制作？"

# content 为包含图片和文本的 multipart 消息列表
messages = [
    {"role": "user", "content": [
        {"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{bstring}", "detail": "high"}}, # 注意：这里将 jpg和 png要匹配文件名
        {"type": "text", "text": user_prompt}
    ]}
]

print(f"模型: {COMPLETION_MODEL}")
print(f"用户提示: {user_prompt}")


模型: step-3
用户提示: 帮我看看这是什么菜，如何制作？


## 执行推理并获取 Reasoning

以下代码将以流式（`stream=True`）方式调用模型，并特别检查 `chunk.choices[0].delta` 中是否包含 `reasoning` 字段，从而将其打印出来。

注意：`reasoning` 字段通常在流的开始部分返回。

In [13]:
time_start = time.time()
print("\n--- 开始流式生成 ---")

# 流式生成
response = client.chat.completions.create(
    messages=messages,
    model=COMPLETION_MODEL,
    stream=True,
)

reasoning_shown = False  # 控制只显示一次 reasoning

for chunk in response:
    delta = chunk.choices[0].delta


    if  hasattr(delta, 'reasoning') and delta.reasoning:
        if not reasoning_shown :        # 只在第一次出现 reasoning 的时候输出
            print(chunk.id) # 输出 traceid
            reasoning_shown = True
            time_firstWord = time.time()
            elapsed_time = time_firstWord - time_start
            print(f"\n首字生成时间: {elapsed_time:.2f} 秒\n")
            print("\n[模型思考过程 Reasoning]:")
        reasoning = chunk.choices[0].delta.reasoning
        print(delta.reasoning, end="", flush=True)
        

    # 正常文本内容
    content = delta.content
    if content:      
        print(content, end="", flush=True)

time_end = time.time()
elapsed_time = time_end - time_start
print("\n\n--- 生成结束 ---")
print("当前模型为:", COMPLETION_MODEL)
print(f"总生成时间: {elapsed_time:.2f} 秒")



--- 开始流式生成 ---
7fe5607f5c8c49b4dbc7b2dce1ae21aa.279c8f2f66a8267118ecac09c815ad6b

首字生成时间: 0.54 秒


[模型思考过程 Reasoning]:
嗯，用户发了一张鱼香肉丝的图片，还问这是什么菜以及怎么制作。从图片来看，确实是经典的鱼香肉丝——猪肉丝搭配胡萝卜、青椒、木耳，红亮油润的酱汁包裹着食材，典型的鱼香风味。

用户可能是刚接触中餐烹饪的新手，或者被这道菜的卖相吸引想尝试复刻。ta没有直接问“鱼香肉丝怎么做”，而是先确认菜名再问做法，说明可能是初次接触这道菜，需要从基础步骤学起。考虑到家庭烹饪的实用性，回复时要突出关键技巧：比如肉丝腌制锁水、鱼香汁比例、快炒火候，这些细节决定成败。

图片里食材切得粗细均匀，说明用户可能注重刀工，但新手没必要追求完美，可以建议“半成品替代”（比如买现成肉丝）。另外鱼香汁的调配是灵魂，得用科学比例（糖醋1:1打底，酱油提鲜增咸，泡椒水增辣增香），还要提醒“淀粉最后勾芡”避免结块。

安全方面要注意：热油飞溅风险（下肉丝时油温高易溅）、切菜防滑（胡萝卜圆滚滚的得按稳）。可以补一句“新手用不粘锅更友好”，但别太啰嗦。最后放张成品图能帮用户对照，不过用户已经发了图，可以省掉。

对了，鱼香肉丝讲究“锅气”，得强调“全程大火快炒”，但别让新手慌神——可以补句“分次下料不着急”，毕竟家庭灶火候不如饭店猛。调味部分把鱼香汁提前调好，避免炒菜时手忙脚乱找调料瓶。

用户没问难度，但作为教学者得预判痛点：比如肉丝柴了怎么办？提前用淀粉蛋清抓匀；汤汁太稀？多放点淀粉水；太酸？糖没放够……这些补救tips可以塞在步骤里，别单独列出来吓人。

最后想，用户可能想知道“鱼香”没有鱼为什么叫这名字？但ta没问，暂时不展开，只在调味时提一句“泡椒是鱼香味灵魂”，点到为止。重点是让用户做出来好吃，文化背景可以下次聊。


这是**鱼香肉丝**，一道经典的川菜（现已风靡全国），以“鱼香”风味为特色——虽无鱼，却通过泡椒、糖、醋等调料模拟出类似烹鱼时的复合香气（咸、甜、酸、辣、鲜）。下面是详细的家常做法：  


### **食材准备**  
- 主料：猪里脊肉 200g（或牛肉，口感更弹）、胡萝卜 1根、青椒 1个、水发黑木耳 50g  
- 调料：泡红椒 2 - 3根